In [1]:
import joblib
from tensorflow.keras.models import load_model

# Load the saved ann model
ann = load_model("H:\\UNT_Classes\\Information retrival\\Project\\saved_models\\ann_model.h5")

# Load the saved rfc model
rfc = joblib.load("H:\\UNT_Classes\\Information retrival\\Project\\saved_models\\rfc_model.joblib")

In [9]:
import bz2
import numpy as np
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):   
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('input/train.ft.txt.bz2')
# test_labels, test_texts = get_labels_and_texts('input/test.ft.txt.bz2')

In [10]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
# test_texts = normalize_texts(test_texts)
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_FEATURES = 12000 
tokenizer =  Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)

In [11]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
inp = [[267.52,1051.0,6.8,128.0,10,1,0,0,'great phone,  my lovely pat has one of the great voices of her generation  i have listened to this mobile for years and i still love it  when i m in a good mood it makes me feel better  a bad mood just evaporates like sugar in the rain  this phone just oozes life  vocals are jusat stuunning and lyrics just kill  one of life s hidden gems  this is a desert isle phone in my book  why she never made it big is just beyond me  everytime i play this  no matter black  white  young  old  male  female everybody says one thing  who was that singing']]
inp1=[inp[0][:-1]]
inp2 = normalize_texts([inp[0][-1]]) 
sc = StandardScaler()
sc.fit(inp1)
X_test = sc.transform(inp1)
y_pred = rfc.predict(X_test)
print(y_pred[0])

inp2 = tokenizer.texts_to_sequences(inp2)
inp2 = pad_sequences(inp2, maxlen=MAX_LENGTH)
preds = ann.predict(inp2)

4


In [15]:
print(preds)

[[0.97276914]]


In [14]:
if(y_pred[0]>3):
    senti = 'Positive'
elif(y_pred[0]==3):
    senti = 'Neutral'
else:
    senti = 'Negative'
print(senti)

Positive
